# 🔗🇨🇳 LangChain核心源代码解读分享

## 💡 这节课会带给你

1. 一起了解 LangChain 的生态
2. 体会阅读 LangChain 源代码
3. 全面了解 LCEL 的用法
4. 全面了解 LangGraph 的用法
5. 将自己的大模型集成到 LangChain
6. 使用 langchain 的不同方案创建智能体

**本节课假设您已经做好了如下准备：**

- 略懂Python编程
- 略懂github
- 略懂langchain开发（最好是已经学完王卓然老师langchain介绍课）

## （一）如何解读 Langchain 的源代码结构

### 1、你应该怎样学习 LangChain ？

- langchain [官方文档](https://python.langchain.com/docs)
- langchain [聊天](https://chat.langchain.com/?llm=anthropic_claude_2_1)
- langchain [源代码](https://github.com/langchain-ai/langchain)
- Github Copilot

### 2、为什么要阅读 LangChain 源码？

- 与最先进的框架一起成长
- 作为了解大模型生态的捷径
- 作为了解服务集成选型的捷径
- 作为了解AI应用生态的捷径
- 作为了解智能体生态的捷径

### 3、LangChain 源代码概览

### 4、LangChain 的生态概览

## （二）先抛出一些 LangChain 的问题

### 1、LangChain核心框架经历了三次较大的迭代

### 2、大模型用不了、新功能未提供

### 3、生态组件的更新不及时、新框架未实现

### 4、组件不兼容：记忆和持久化在文档中的矛盾？


### 5、智能体论文和实践非常快

## （三）从零开始集成大模型到 Langchain 实例

### 1、如何集成智谱AI大模型？

### 2、如何集成你训过的开源大模型？

## （四）全面拆解 LCEL 能力

### 1、如何用 Runnable 组件搭积木？

- Runnable 组件的 8 个方法
- 自定义Runnable（invoke / stream / astream）
- invoke 原理
- schema 探查
- config

### 2、LCEL 比 遗留 Chain 多哪些优势？

- 那些被挽留的 Chain ...
- 这些 Chain 局限性在究竟哪里？
- LCEL 构建的替代 Chain

### 3、如何用 LCEL 定义智能体？

- 工具：定义一个简单工具
- 智能体：
    - Tools-Calling 智能体
    - ReAct 智能体
- 执行器：AgentExecutor
- 调用逻辑：从源代码看AgentExecutor的调用逻辑？

### 4、当前推荐使用的 LangChain 体系？

- 框架
    - LCEL
    - langgraph
    - langserve
- 提示语
- 大模型
- 输出解析
- 工具
    - 工具集
    - 当作 chain 在 langgraph 中使用
- 智能体
    - 智能体定制（推荐使用langgraph）
    - 各种智能体论文范例
- 构建你自己的 langchain 框架

## （五）全面拆解 LangGraph 能力

### 1、LangGraph 比 LCEL 多了什么？

### 2、如何使用 LangGraph 定义智能体？

## （六）综合场景

### 1、如何享用 Langserve 带来的便利？

### 2、如何实现各种场景下流式输出？

- 链的流
- 工具的流

### 3、如何用 AgentExcutor 再现《手撕AutoGPT》？

### 4、如何用 LangGraph 再现《手撕AutoGPT》？